Analysis

In [122]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]

In [123]:
import pandas as pd

In [526]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl=cl.sort_values('n').reset_index(drop=True)

DD analysis

In [527]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [528]:
cl[10:11]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
10,1,1,0,0,625,"[-5, -9, 7]","[5, 15]",7,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, -4, 22]}]",{'2': [-18]},"[2, -4, 7, 9, -18, -18, 22]",[-18],[-18],0,0


In [575]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0):
    s=-(ν+m)
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,XD=False):
    m=0
    if XD:
        ms=[ x for x in l if l.count(x)==3 ]
        if ms:
            m=ms[0]

    return len( get_massless_fermions(l,ν,m) )

In [576]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [592]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)

[9]

In [577]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,XD=False)==2

True

In [578]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,XD=True)==1

True

In [579]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [580]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [584]:
def massfer(row,label='DarkDirac',nmax=10):
    if label=='DarkDirac':
        if row.get(label)>0:
            cdd=[]
            for ν in row.get('DD'):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν)  )
            return min(cdd)
        else:
            return nmax

In [585]:
get_massless_fermions([2, -4, 7, 9, -18, -18, 22],ν=-18)

[2, 7]

In [590]:
cl[:1542].apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')

0        0
1        0
2        0
3        0
4       10
5        3
6        3
7        2
8        3
9        3
10       2
11       3
12       3
13       3
14       3
15       2
16       3
17       3
18       2
19       3
20       3
21       3
22       2
23       2
24       2
25       2
26      10
27       2
28       3
29      10
30      10
31      10
32       1
33      10
34       1
35       3
36      10
37       3
38       3
39       3
40       3
41       1
42       3
43       1
44       3
45       1
46       3
47       2
48       3
49       3
        ..
1492     4
1493     5
1494     5
1495     5
1496     4
1497     5
1498     5
1499     5
1500     5
1501     4
1502     4
1503     2
1504     3
1505     5
1506     5
1507     5
1508     5
1509     5
1510     5
1511     5
1512     5
1513     5
1514     4
1515     5
1516     2
1517     4
1518     5
1519     4
1520     3
1521     3
1522     5
1523     5
1524     4
1525     3
1526     4
1527     4
1528     5
1529     5
1530     4
1531     5

In [593]:
cl[1542:1543]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
1542,1,0,0,0,8,"[-2, -5, -2, 3]","[-4, -2, -4]",9,"[{'D→nu_R+fi+fj': [-9, 2, 7]}]",{'4': [2]},"[2, 2, 2, 2, 7, -8, -9, -11, 13]",0,0,0,0


In [591]:
cl[1537:1538]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
1537,1,0,0,0,432,"[6, -1, 7, 1]","[12, 0, 9]",9,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",{'2': [-3]},"[2, -3, -3, -5, 9, -13, 15, 16, -18]",[-3],0,0,0


In [471]:
cl[:3]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
0,2,0,0,0,2,"[-5, 0]","[1, -1]",6,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",{'2': [13]},"[2, -3, -10, 13, 13, -15]",[13],0,0,0
1,2,0,0,0,7749,"[4, 7]","[21, 18]",6,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",{'2': [25]},"[3, -4, -21, 25, 25, -28]",[25],0,0,0
2,2,0,0,0,1,"[-4, 0]","[-1, 1]",6,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",{'2': [17]},"[3, -5, -12, 17, 17, -20]",[17],0,0,0


### Tree level neutrino Dirac masses

In [416]:
l=get_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13,m=0)
[ ll for ll in l if ll+ν-s==0]

[]

In [417]:
[ ll for ll in l if ll-ν+s==0]

[]